In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [4]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [7]:
model = models.Sequential()
model.add(ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(264, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

94668760/94668760 [==============================] - 0s 0us/step


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 264)               26493192  
                                                                 
 dense_1 (Dense)             (None, 128)               33920     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 50092041 (191.09 MB)
Trainable params: 50046601 (190.91 MB)
Non-trainable params: 45440 (177.50 KB)
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

Epoch 1/100
20/20 [==============================] - 98s 2s/step - loss: 1.8058 - accuracy: 0.7781 - val_loss: 47.5433 - val_accuracy: 0.4938
Epoch 2/100
20/20 [==============================] - 8s 352ms/step - loss: 0.2667 - accuracy: 0.9203 - val_loss: 245.6185 - val_accuracy: 0.6062
Epoch 3/100
20/20 [==============================] - 8s 357ms/step - loss: 0.3440 - accuracy: 0.8734 - val_loss: 48154.4570 - val_accuracy: 0.4938
Epoch 4/100
20/20 [==============================] - 9s 365ms/step - loss: 0.2806 - accuracy: 0.9047 - val_loss: 1282183.0000 - val_accuracy: 0.4938
Epoch 5/100
20/20 [==============================] - 8s 353ms/step - loss: 0.2338 - accuracy: 0.9266 - val_loss: 781249.1875 - val_accuracy: 0.4938
Epoch 6/100
20/20 [==============================] - 8s 367ms/step - loss: 0.1974 - accuracy: 0.9359 - val_loss: 850.1205 - val_accuracy: 0.5125
Epoch 7/100
20/20 [==============================] - 8s 356ms/step - loss: 0.1639 - accuracy: 0.9391 - val_loss: 1060.5801 -

In [11]:
model.save('/content/drive/MyDrive/resnet50v2sequence_indoor_outdoor_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
model.evaluate(val_ds)

5/5 [==============================] - 2s 105ms/step - loss: 0.7705 - accuracy: 0.8687


[0.7705132961273193, 0.8687499761581421]

In [13]:
def process_directory(model, prediction_dir):
    for img in os.listdir(prediction_dir):
        try:
            img_path = os.path.join(prediction_dir, img)

            img = image.load_img(img_path, target_size=(img_width, img_height))
            img_array = image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)

            img_array /= 255.0

            predictions = model.predict(img_array)
            predicted_class = 'outdoor' if predictions[0] >= 0.5 else 'indoor'
            probability = predictions[0]

            print(f'{img} - Predicted Class: {predicted_class}, Probability: {probability}')

        except Exception as e:
            print(f'Exception processing {img}: {str(e)}')

In [14]:
class_labels = None

In [ ]:
outdoor_prediction_dir = '/content/drive/MyDrive/IndoorOutdoorDataset/indoor'
process_directory(model, outdoor_prediction_dir)

In [ ]:
outdoor_prediction_dir = '/content/drive/MyDrive/IndoorOutdoorDataset/outdoor'
process_directory(model, outdoor_prediction_dir)

In [ ]:
predicted_prediction_dir = '/content/drive/MyDrive/predicted'
process_directory(model, predicted_prediction_dir)